# The aim of this notebooks is to increse the word coverage for embedding and see if this increases the accuracy.

## Things Learned.

### Its not good to apply the same preprocessing steps to every dataset 

### while using embeddins the preprocessing needs to try to get the dataset as close to the embedding words used to increase the words coverage.


1. When the trainnig is set to true for embedding layer it reduces the number of epochs need by more than half.
2. Blindly apply stemming , lemmatization can reduce the word coverage.
3. Applying spacing for punctuations increases the coverafe the most.
4. Word Correction was also another important step to increase coverage.
5. I also tried contraction correction but this didnt help mush.
6. Taking care of extra spaces is also an important step in preprocessing.
7. Removing Html tags
8. Removing punctuations except [' , . ,-].
     
     8.a .,- needs to be replaced by '' and not ' '.
     
     8.b (optional) If you want to keep punctuations.
        One approach is to only keep the one's common in embeddings and dataset.

9. Its better to set the training 


# Results:

1. With initial preprocessing step the embedding coverage was 97% and test score was 83.
2. With punct. preprocessing and Word Correction we increased the coverage to 99.% and test score of 86.

So. word processing did help.




In [1]:
import pickle as pk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm
from textblob import TextBlob
from sklearn.metrics import accuracy_score

print(os.listdir())

import nltk
# nltk.download('punkt')
# nltk.download('stopwords')



import gc

['.ipynb_checkpoints', 'clr.py', 'Completed Hacktons', 'Detecting Outlinears.ipynb', 'df_questions', 'embedding', 'Feature Secection.ipynb', 'Glove Preprocessing .ipynb', 'Increasing word coverage of Embedding .ipynb', 'Learning Rate Schedularas.ipynb', 'MNIST PCA.ipynb', 'mnist.npz', 'PCA.ipynb', 'processed_train_data', 'Simalarity.ipynb', 'Spell Checker using Word2vec.ipynb', 'TextDataTemplate.ipynb', 'to do.txt', 'Unballanced Classes .ipynb', 'untitled.txt', 'Visualizing Word Embeddings.ipynb', 'word2vec-nlp-tutorial', '__pycache__']


In [2]:

#helper functions 

from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

import operator
####helper functions 
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab



def get_unknown_words(text,word_matrix):
    
    vocab_raw = build_vocab(text)
    unk_words_raw = check_coverage(vocab_raw,word_matrix)
    
    return unk_words_raw



def  get_matrix(tokenizer,emb_dim,embeddings):

  embedding_matrix = np.zeros((len(tokenizer.word_index) + 1,emb_dim))

  count  = 0
  for word,index in tokenizer.word_index.items():

      vec = embeddings.get(word)
      
      if vec is not None:
        embedding_matrix[index] = vec
      else:
          count += 1

  print(f'{count/len(tokenizer.word_index)} empty')

  return  embedding_matrix   


Using TensorFlow backend.


In [3]:
#loading data 

train_dataset_path = 'word2vec-nlp-tutorial/labeledTrainData.tsv.zip'
test_dataset_path = 'word2vec-nlp-tutorial/testData.tsv.zip'


train_dataset = pd.read_csv(train_dataset_path,sep = '\t')

train_dataset.head()


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
train_dataset.shape

(25000, 3)

 Generating Test Sentiments 
Cheat Code 

In [6]:
test_dataset = pd.read_csv(test_dataset_path,sep = '\t')

def get_labels(x):

  return int(int(x.split('_')[-1]) >=5)


test_dataset['sentiment'] = test_dataset['id'].map(get_labels)



In [7]:
test_dataset.head()

,id,review,sentiment
0,12311_10,Naturally in a film who's main themes are of m...,1
1,8348_2,This movie is a disaster within a disaster fil...,0
2,5828_4,"All in all, this is a movie for kids. We saw i...",0
3,7186_2,Afraid of the Dark left me with the impression...,0
4,12128_7,A very accurate depiction of small time mob li...,1


In [8]:
#checking class balance 

print(train_dataset['sentiment'].value_counts())

print(test_dataset['sentiment'].value_counts())



1    12500
0    12500
Name: sentiment, dtype: int64
1    12500
0    12500
Name: sentiment, dtype: int64


#### EDA 

1. check the number of words 
2. check the number of punctuations 
3. Digits 
4. other special chatarters.
5. get the max , min , avg. length


In [9]:
full_text = list(train_dataset['review']) + list(test_dataset['review'])

In [10]:
pd.Series(full_text)

0        With all this stuff going down at the moment w...
1        \The Classic War of the Worlds\" by Timothy Hi...
2        The film starts with a manager (Nicholas Bell)...
3        It must be assumed that those who praised this...
4        Superbly trashy and wondrously unpretentious 8...
                               ...                        
49995    Sony Pictures Classics, I'm looking at you! So...
49996    I always felt that Ms. Merkerson had never got...
49997    I was so disappointed in this movie. I am very...
49998    From the opening sequence, filled with black a...
49999    This is a great horror film for people who don...
Length: 50000, dtype: object

In [11]:
vocab = build_vocab(pd.Series(full_text))

len(vocab)

449675

In [12]:
import string 

words = []
numbers = []
punctuations = []
need_cleaning = []
chars = []

print('number of unique words')

for word in vocab.keys():

    if word.isdigit():
      numbers.append(word)
    elif len(word) == 1 and word in string.punctuation:
      punctuations.append(word)
    elif len(word) == 1:
      chars.append(word)
    else:
       words.append(word)    






number of unique words


In [13]:
print(f'number of words {len(words)}')
print(f'number of digits {len(numbers)}')
print(f'number of punct {len(punctuations)}')
print(f'number of chars {len(chars)}')


number of words 449008
number of digits 563
number of punct 31
number of chars 73


In [14]:
def print_data(x):

  for i in x:
    print(i, end = ' ')

print_data(punctuations)    

. - , : ( & ? ) = / * ! % \ ~ # " ' > + $ @ } ` ; { [ _ < ] ^ 

In [15]:
print_data(chars)

i a I A R  L Q B S o n G O D v – u X V s P H ® y J K  T E U N Z x b r f C e F à W ½ d l Y M c h z m t w ¨ p   À ¾ » · é Ð ö g  j ý è k ­ á Ö 

In [16]:
def is_chars(x):
    
    
    if ord(x)>=65 and ord(x)<=90 or ord(x) >=97 and ord(x)<=122:
        return True
    
    else:
        return False

In [17]:
special_chars_to_remove = []

for i in chars:
    
    if not is_chars(i):
        special_chars_to_remove.append(i)
        
        
        
special_chars_to_remove        

['\x96',
 '–',
 '®',
 '\x97',
 'à',
 '½',
 '¨',
 '\uf0b7',
 '\x95',
 'À',
 '¾',
 '»',
 '·',
 'é',
 'Ð',
 'ö',
 '\x80',
 'ý',
 'è',
 '\xad',
 'á',
 'Ö']

 Ckecking out the words 

In [18]:
print(f'There are {len(words)} words.')

There are 449008 words.


In [19]:
#lower casing all words 


lower_cased_words = list(map(lambda x : x.lower(),words))
final_words = set(lower_cased_words)

print(f'final word count after lower casing {len(final_words)}')

final word count after lower casing 400932


In [ ]:
ord('z')

In [20]:
#find words with punctuations or numbers in them 

def checker(x):

  for i in string.punctuation:

    if i in x:
      return True

  return False


improper_words = []

for word in final_words:

    if checker(word):improper_words.append(word)

        
        
        


In [21]:
len(improper_words)

317311

In [114]:
#using extended punct


# puncts = [',', '"', ':', ')', '(', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
#  '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
#  '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
#  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
#  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]


puncts = ['´',',', '"','.', ':', ')', '(', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '●', 'â', '►', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]



punct_to_remove = []



def checker(x):

  for i in  puncts:

    if i in x:
      punct_to_remove.append(i)
      return True

  return False


improper_words = []

for word in final_words:

    if checker(word):improper_words.append(word)



len(improper_words)


267924

found 614 more words using entended punctuations list 

In [115]:
special_chars_to_remove

['\x96',
 '–',
 '®',
 '\x97',
 'à',
 '½',
 '¨',
 '\uf0b7',
 '\x95',
 'À',
 '¾',
 '»',
 '·',
 'é',
 'Ð',
 'ö',
 '\x80',
 'ý',
 'è',
 '\xad',
 'á',
 'Ö']

In [116]:
extended_punct = '|\\'.join(puncts+special_chars_to_remove)

print(extended_punct)

re.sub(extended_punct,' ','father(henry?øØ')

´|\,|\"|\.|\:|\)|\(|\!|\?|\||\;|\$|\&|\/|\[|\]|\>|\%|\=|\#|\*|\+|\\|\•|\~|\@|\£|\·|\_|\{|\}|\©|\^|\®|\`|\<|\→|\°|\€|\™|\›|\♥|\←|\×|\§|\″|\′|\Â|\█|\½|\à|\…|\“|\★|\”|\●|\â|\►|\¢|\²|\¬|\░|\¶|\↑|\±|\¿|\▾|\═|\¦|\║|\―|\¥|\▓|\—|\‹|\─|\▒|\：|\¼|\⊕|\▼|\▪|\†|\■|\’|\▀|\¨|\▄|\♫|\☆|\é|\¯|\♦|\¤|\▲|\è|\¸|\¾|\Ã|\⋅|\‘|\∞|\∙|\）|\↓|\、|\│|\（|\»|\，|\♪|\╩|\╚|\³|\・|\╦|\╣|\╔|\╗|\▬|\❤|\ï|\Ø|\¹|\≤|\‡|\√|\|\–|\®|\|\à|\½|\¨|\|\|\À|\¾|\»|\·|\é|\Ð|\ö|\|\ý|\è|\­|\á|\Ö


'father henry ø '

In [117]:

#from the above preprocessing punct list i have removed ', and .


def preprocess(x):
    
    regex = re.compile('<.*?>')
    input =  re.sub(regex, ' ',x)
    
    #removing punctuations.
    input = re.sub(extended_punct,' ',input)
    

    
    
    
    return input.lower() 


processed_text = pd.Series(full_text).map(preprocess)


In [118]:
import string 

words = []
numbers = []
punctuations = []
need_cleaning = []
chars = []


vocab = build_vocab(processed_text)


print('number of unique words')

for word in vocab.keys():

    if word.isdigit():
      numbers.append(word)
    elif len(word) == 1 and word in string.punctuation:
      punctuations.append(word)
    elif len(word) == 1:
      chars.append(word)
    else:
       words.append(word)    

    
    
    
    

print(f'number of words {len(words)}')
print(f'number of digits {len(numbers)}')
print(f'number of punct {len(punctuations)}')
print(f'number of chars {len(chars)}')

number of unique words
number of words 156380
number of digits 760
number of punct 2
number of chars 31


In [119]:
punctuations

['-', "'"]

# Applying Preporcessing to train and text

In [29]:
train_dataset['processed'] = train_dataset['review'].map(preprocess)
test_dataset['processed'] = test_dataset['review'].map(preprocess)

# Loading Glove Embeddings

In [30]:
#load embeddings 

glove_filepath = 'embedding/glove.6B/glove.6B.300d.txt'
def load_embed(file):
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
  
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

word_matrix = load_embed(glove_filepath)

In [31]:
len(word_matrix)

400000

## Checking Coverage 

In [120]:
unk_words = check_coverage(vocab,word_matrix)

print(f'no of unk words {len(unk_words)}')

Found embeddings for 50.35% of vocab
Found embeddings for  97.07% of all text
no of unk words 78041


In [34]:
# unk_words

### So now we have 2 options
1. Either use Contrctions 
2. split the words in 2 halfs eg [it  ,'s]
3. For the . we need to replce by ''

## But First lets check the perforamce of the model on
1. the embedding layer with no. pre-trained embeddings
2. using glove 300d embedding training == False
3. using glove embeddings training == True


 The first preprocessing function did not contain any preprocessning needed to increase word embeddings


In [35]:
processed_text

0        with all this stuff going down at the moment w...
1         the classic war of the worlds   by timothy hi...
2        the film starts with a manager  nicholas bell ...
3        it must be assumed that those who praised this...
4        superbly trashy and wondrously unpretentious 8...
                               ...                        
49995    sony pictures classics  i'm looking at you  so...
49996    i always felt that ms. merkerson had never got...
49997    i was so disappointed in this movie. i am very...
49998    from the opening sequence  filled with black a...
49999    this is a great horror film for people who don...
Length: 50000, dtype: object

### Getting the Input Data Ready
1. train the tokenizer make user that the filers is set to none.
2. Convert the input to sequence 
3. pad them (the padding length maxlen is a hyperparameter.)
4. get the labels in proper form 
5. split the data. the splitting ratio 80-20 is also a hyperparameter.

In [ ]:
#Word embeddinsg 
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower = True,filters = '')

tokenizer.fit_on_texts(processed_text)


vocab_size = len(tokenizer.word_index) + 1;vocab_size


print(f'vocab size {vocab_size}')

print('tokenizing')

X = tokenizer.texts_to_sequences(train_dataset['processed'])

In [ ]:
processed_text.map(lambda x : len(x.split())).plot(kind = 'hist')

In [ ]:
maxlen = 100
from keras.preprocessing.sequence import pad_sequences

print(f'padding with max len of {maxlen}')

X_pad = pad_sequences(X,maxlen)

In [63]:
Y_data = train_dataset['sentiment']

Y_data.shape

(25000,)

In [64]:
from keras.utils import to_categorical

Y_data = to_categorical(Y_data,num_classes=2)
Y_data.shape

(25000, 2)

In [ ]:
from sklearn.model_selection import train_test_split 


trainx,testx,trainy,testy = train_test_split(X_pad,Y_data,test_size = 0.2,random_state = 101)

trainx.shape

In [ ]:
vocab_size

Using the embedding layer I tried using both with train = False and True.

With False the training time reduces but it takes more number of epochs / steps about 10 to get 74. While setting 
it to True we get a 70 % above accuracy in the 1st epoch and then the model starts overfitting.


I have also no



In [74]:
#LSTM netrow
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

def get_model(vocab_size,weights = None,dim = 128,train = False,maxlen = 100):
  

    model = Sequential()
    model.add(Embedding(vocab_size,
                      dim,
                      weights= weights,
                      input_length=maxlen,
                      trainable=train))

    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(100))

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.8))

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.8))

    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(learning_rate = 0.001),
                  metrics = ['accuracy',f1])

    return model


model  = get_model(vocab_size,dim = 300,train = True)
model.summary()


W0722 12:54:45.573032  5808 nn_ops.py:4372] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0722 12:54:45.626021  5808 nn_ops.py:4372] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          31837200  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1024)              103424    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0

In [ ]:
trainx.shape,trainy.shape

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler,ReduceLROnPlateau

# es = EarlyStopping(monitor= 'val_accuracy',min_delta = 0.0001,patience=5,restore_best_weights=True,verbose = 1)
# rop = ReduceLROnPlateau(monitor = 'loss')


history = model.fit(trainx,trainy,
                    epochs = 3,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    )

In [77]:

def get_test_score():

    test_data = tokenizer.texts_to_sequences(test_dataset['processed'])
    test_pad = pad_sequences(test_data,maxlen=100)

    preds = model.predict(test_pad)

    

    print(f"test score is {accuracy_score(np.argmax(preds,axis = 1),test_dataset['sentiment'])}")
    
    
get_test_score()    
    
    
    

test score is 0.857


In [ ]:
del model
# del df_questions
gc.collect()

# Using Word Glove 

In [ ]:
weights = get_matrix(tokenizer,emb_dim = 300,embeddings=word_matrix)
weights.shape

In [ ]:
del word_matrix
# del df_questions
gc.collect()

In [ ]:

model = get_model(vocab_size,weights=[weights],train = False,dim=300)


model.summary()

history = model.fit(trainx,trainy,
                    epochs = 3,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    
                    )

In [ ]:
get_test_score()

# with train = True

In [ ]:

model = get_model(vocab_size,weights=[weights],train = True,dim=300)


model.summary()

history = model.fit(trainx,trainy,
                    epochs = 3,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    
                    )

In [ ]:
get_test_score()

In [ ]:
del model
gc.collect()

# Findings

In [ ]:
data = {'method':['embd','glove_F','glove_T'],
        'epochs':[3,3,3],
        'Train Score':[96,81,91],
        'Val Score':[83,83.73,86],
        'Test Score':[82.73,83.41,85],
        'TTPEim':[3,1,3]
       }

pd.DataFrame(data)

# Applying the second stage of preprocessing for Glove 

### Main Aim over here is to increase the coverage and see if this increses our accuracy.



### Checking out of context words:

In [ ]:

word_matrix = load_embed(glove_filepath)

In [ ]:
len(vocab)

In [36]:
oov_words = check_coverage(vocab,word_matrix)

Found embeddings for 37.04% of vocab
Found embeddings for  92.50% of all text


In [37]:
oov_words[0:4]

[("it's", 33014), ("don't", 16530), ('it.', 12995), ('movie.', 12339)]

we can try doing 
1. check if splitting  words like it's to it 's helps. 
2. check if converting it to it is helps.
3. do spelling correction on out of vocab words only and see if they have 

# Using approach 1

In [123]:
#preprocessing pipeline 
import re


puncts = ['´',',', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]


to_replace = '´'


def clean_text(x):
    
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    
    
    #will convert ´ to '
    if to_replace in x:
        x = x.replace(to_replace,"'")
        
    
    
    #willl convert e.g to eg
    if '.' in x:
        x = x.replace('.','')
        
  
        
        
    return x



    




# Clean the text
cleaned_text = processed_text.apply(clean_text)

print('preprocessing done...')

unk_words = get_unknown_words(cleaned_text,word_matrix)



preprocessing done...
Found embeddings for 74.20% of vocab
Found embeddings for  99.64% of all text


In [125]:
print(f'no. of unk words remaining {len(unk_words)}')

no. of unk words remaining 26220


we have reduced out of vocab words from 132853 to 39567

# Using option 2 taking care of contractions

In [40]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [41]:

def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known



print("- Known Contractions -")
print("   Glove :")
print(known_contractions(word_matrix))

- Known Contractions -
   Glove :
["'cause", "ma'am", "o'clock"]


In [42]:
vocab_raw = build_vocab(processed_text)
contaction_list = known_contractions(vocab_raw)

print(f'no. of contractions found is {contaction_list}')


no. of contractions found is ["ain't", "aren't", "can't", "'cause", "could've", "couldn't", "didn't", "doesn't", "don't", "hadn't", "hasn't", "haven't", "he'd", "he'll", "he's", "how'd", "how's", "i'd", "i'd've", "i'll", "i'm", "i've", "isn't", "it'd", "it'll", "it's", "let's", "ma'am", "might've", "mightn't", "must've", "mustn't", "needn't", "o'clock", "shan't", "she'd", "she'll", "she's", "should've", "shouldn't", "so's", "this's", "that'd", "that's", "there'd", "there's", "here's", "they'd", "they'll", "they're", "they've", "to've", "wasn't", "we'd", "we'll", "we're", "we've", "weren't", "what'll", "what're", "what's", "what've", "when's", "where'd", "where's", "who'll", "who's", "who've", "why's", "won't", "would've", "wouldn't", "y'all", "you'd", "you'll", "you're", "you've"]


In [43]:


def clean_contractions(text, mapping):
    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    
    #does the replacing 
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    
    return text

text_no_contractions = processed_text.map(lambda x : clean_contractions(x,contraction_mapping))

unk_words = get_unknown_words(text_no_contractions,word_matrix)

print(f'no. of unk words remaining {len(unk_words)}')

Found embeddings for 37.07% of vocab
Found embeddings for  93.97% of all text
no. of unk words remaining 132677


SO the contractions did not work must just incresed the coverage by 1% from 97 to 98

We will go forward with approach 1

In [127]:
# Clean the text
cleaned_text = processed_text.apply(clean_text)

print('preprocessing done...')

unk_words = get_unknown_words(cleaned_text,word_matrix)


preprocessing done...
Found embeddings for 74.20% of vocab
Found embeddings for  99.64% of all text


In [128]:
unk_words[0:4]

[('hadn', 540), ('imho', 79), ('gypo', 60), ('feinstone', 49)]

# In the last step we will be correcting spellings 

In [46]:
'apple' in 'apple products have good hardware'

True

In [ ]:
# A faster approach is to createa a dict of {'mis-spelled word':'correct-spelled word'} and only replace those 
#words that change 

In [48]:
# import tqdm



# #takes 2 long.

# # def correct_spelling(x,unk_owrds):
    
    
# #     for word,count in unk_words:
        
# #         if word in x:
# #             return str(TextBlob(x).correct())
    
    
    
# #     return x


# correct_dict = {}


# for word,count in unk_words:
    
#     print(word,TextBlob(word).correct())








# # print(f'no. of unk_words. before correction. {len(unk_words)}')

# # corrected_text = cleaned_text.map(lambda x : correct_spelling(x,unk_words))
    
# # unk_words = get_unknown_words(corrected_text,word_matrix)
    
# # print(f'no. of unk. owrds after correction {len(unk_words)}')    
    

### A better method is stem,lemmatize and correct the word when creating the embedding matrix but this method doesnt change the word in the vocab code from kaggle 

In [49]:
#imports 
from __future__ import absolute_import, division

import os
import time
import numpy as np
import pandas as pd
import gensim
from tqdm import tqdm


### stemmers 


from nltk.stem import PorterStemmer
ps = PorterStemmer()


from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()


from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")


import gc

import sys
from os.path import dirname

import spacy

In [50]:
path = 'GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format('embedding/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', 
                                                        binary=True)

In [51]:
words = model.index2word

print('loaded...')

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

print('done..')


# Use fast text as vocabulary
def words(text): return re.findall(r'\w+', text.lower())
def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or [word])
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def singlify(word):
    
    return "".join([letter for i,letter in enumerate(word) if i == 0 or letter != word[i-1]])

loaded...
done..


In [52]:
del model
gc.collect()

20

# Creating the correct Dict

In [53]:
correct_dict = {}


for word,count in unk_words:
    correct_word = correction(word)
    
    if correct_word != word:
        correct_dict[word] = correct_word



In [129]:
'gypo' in correct_dict

True

In [130]:


def correct_mapping(text,mapping):
    
    #does the replacing 
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    
    return text
    
    

print(f'no. of unk_words. before correction. {len(unk_words)}')
    
    
corrected_text = cleaned_text.map(lambda x : correct_mapping(x,correct_dict))

    
unk_words = get_unknown_words(corrected_text,word_matrix)
    
print(f'no. of unk. owrds after correction {len(unk_words)}')    

no. of unk_words. before correction. 26220
Found embeddings for 81.90% of vocab
Found embeddings for  99.75% of all text
no. of unk. owrds after correction 16928


## Applying Approach 1 . and Correction to Train and Test data

In [56]:
## method to call clean_text ,correct_mapping    

train_dataset['processed'] = train_dataset['processed'].map(clean_text)
train_dataset['processed'] = train_dataset['processed'].map(lambda x : correct_mapping(x,correct_dict))


In [57]:
test_dataset['processed'] = test_dataset['processed'].map(clean_text)
test_dataset['processed'] = test_dataset['processed'].map(lambda x : correct_mapping(x,correct_dict))


### This has reduced the words from 26000 to 16000 

 At last we are checking if stemming  and lemmatizing the text increases the coverage

# Code for lemmatization

In [131]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk import word_tokenize



def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
              'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(text): 
    wnl = WordNetLemmatizer()
    
    # Text input is string, returns lowercased strings.
    ls = list(wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tag(word_tokenize(text)))

    str = ''

    for i in  ls:
          str += i.lower() + ' '

    return str  



lemmatize_sent('running')

'run '

In [61]:

stem = []
correct = []
lemmatize = []
no_embd = []


def load_glove(tokenizer,embeddings_index,emb_dim = 100):

    '''
    word is check 
    1. the actual word 
    2. lower case and check
    3. upper case and check
    4. captalize and check
    5. Stem and check
    6. lemma and check
    7. correct spelling and check.
    '''
    known_words = 0
    unk_words = 0


    embedding_matrix = np.zeros((len(tokenizer.word_index)+ 1,emb_dim)) - 1.0

    count  = 0

    for key,index in tqdm(tokenizer.word_index.items()):

        word = key

        embedding_vector = embeddings_index.get(word)


        #checking if word is present.
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            known_words += 1
            continue

        #check if lower case of the word is present.
        word = key.lower()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            known_words += 1
            continue

        #cheking if the upper case of the word is present.
        word = key.upper()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            known_words += 1
            continue

        #checking if capital word is present.
        word = key.capitalize()
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            known_words += 1
            continue

        '''
        Three steppers are  used 

        ps -> PorterStemmer 
        ls -> lancaster 
        sb->snowball stemmer 

        need to check on this as well

        
        '''    



        word = ps.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            stem.append((word,key))
            known_words += 1
            continue


        word = lc.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            stem.append((word,key))
            known_words += 1
            continue


        word = sb.stem(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            stem.append((word,key))
            known_words += 1
            continue

        '''
        Using Lemmatization
        
        '''    


        word = lemmatize_sent(key)
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[word_dict[key]] = embedding_vector
            lemmatize.append((word,key))
            known_words += 1
            continue

        ''''
        Checking for spellling mistakes 
        
        '''    



        if len(key) > 1:
            word = correction(key)
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
                correct.append((word,key))
                known_words += 1
                continue

                
        #unknown words.    
        # embedding_matrix[word_dict[key]] = unknown_vector        
        unk_words += 1
        no_embd.append(key)
    
    

    print(known_words / len(tokenizer.word_index))

    return embedding_matrix





In [133]:
#Word embeddinsg 
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters = '')

tokenizer.fit_on_texts(corrected_text)


vocab_size = len(tokenizer.word_index) + 1;vocab_size

print('tokenizing')

X = tokenizer.texts_to_sequences(train_dataset['processed'])



maxlen = 100
from keras.preprocessing.sequence import pad_sequences

print(f'padding with max len of {maxlen}')

X_pad = pad_sequences(X,maxlen)


from sklearn.model_selection import train_test_split 


trainx,testx,trainy,testy = train_test_split(X_pad,Y_data,test_size = 0.2)

trainx.shape


tokenizing


 79%|███████▉  | 84338/106123 [00:40<00:19, 1127.22it/s]

padding with max len of 100


(20000, 100)

In [134]:
len(tokenizer.word_index),vocab_size

(94468, 94469)

In [135]:

print('done building vocab')
weights = load_glove(tokenizer,word_matrix,emb_dim = 300)

done building vocab



 87%|████████▋ | 82564/94468 [00:16<00:07, 1693.37it/s]


100%|██████████| 94468/94468 [00:23<00:00, 4048.17it/s]

0.8578142863191769


## Embeddings found because of stemming 

In [136]:
print(f'no. of words found becz. of stemming are {len(stem)}')

no. of words found becz. of stemming are 10565


# Embeddings found because of Lemmatization

In [137]:
print(f'no. of words found becz. of lemmatization are {len(lemmatize)}')

no. of words found becz. of lemmatization are 0


# Embeddings found because of correction 

In [138]:
print(f'no. of words found becz. of lemmatization are {len(correct)}')

no. of words found becz. of lemmatization are 1809


Why did we find more words in correction ?
Ealier we only took are of words found in the found in unk_words

In [ ]:
correct

A lot of these last correction are just spaces we should probably modify our preprocessing pipeline to include this spaces or create a regax that removes extra spaces

# Words with no. EMbeddings.

In [71]:
print(len(no_embd))
# no_embd

24238


In [72]:
# import gc
# del model,tokenizer
gc.collect()

100

# Training the Models with Weights having 99% word coverage

In [75]:


model = get_model(vocab_size,[weights],300,train = False)

model.summary()

history = model.fit(trainx,trainy,
                    epochs = 10,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    )

W0722 12:55:22.823021  5808 nn_ops.py:4372] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0722 12:55:22.875717  5808 nn_ops.py:4372] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          31837200  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              103424    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)             

W0722 12:55:23.400757  5808 nn_ops.py:4372] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


20000/20000 [==============================] - 51s 3ms/sample - loss: 0.5956 - accuracy: 0.6724 - f1: 0.6727 - val_loss: 0.4602 - val_accuracy: 0.7903 - val_f1: 0.7907
Epoch 2/10
20000/20000 [==============================] - 49s 2ms/sample - loss: 0.4869 - accuracy: 0.7749 - f1: 0.7749 - val_loss: 0.4191 - val_accuracy: 0.8066 - val_f1: 0.8063
Epoch 3/10
20000/20000 [==============================] - 53s 3ms/sample - loss: 0.4389 - accuracy: 0.8012 - f1: 0.8012 - val_loss: 0.4177 - val_accuracy: 0.8122 - val_f1: 0.8122
Epoch 4/10
20000/20000 [==============================] - 57s 3ms/sample - loss: 0.4049 - accuracy: 0.8202 - f1: 0.8203 - val_loss: 0.3895 - val_accuracy: 0.8261 - val_f1: 0.8262
Epoch 5/10
20000/20000 [==============================] - 55s 3ms/sample - loss: 0.3783 - accuracy: 0.8338 - f1: 0.8339 - val_loss: 0.3657 - val_accuracy: 0.8349 - val_f1: 0.8349
Epoch 6/10
20000/20000 [==============================] - 60s 3ms/sample - loss: 0.3613 - accuracy: 0.8452 - f1: 0.8

In [76]:
get_test_score()

NameError: name 'get_test_score' is not defined

### 3% increase form 83 to 86

In [ ]:


model = get_model(vocab_size,[weights],300,train = True)

model.summary()

history = model.fit(trainx,trainy,
                    epochs = 3,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    )

In [ ]:
get_test_score()

# L2 Normalizing the weights 

In [81]:
from sklearn.preprocessing import Normalizer

norm = Normalizer()
print(f'shape of weight {weights.shape}')

print('normalizing...')

weightsnorml = norm.fit_transform(weights)

print(f'shape of weight {weightsnorml.shape}')



shape of weight (106124, 300)
normalizing...
shape of weight (106124, 300)


In [88]:

# model = get_model(vocab_size,[weightsnorml],300,train = False)

# model.summary()

history = model.fit(trainx,trainy,
                    epochs = 10,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    )

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/sample - loss: 0.3703 - accuracy: 0.8381 - f1: 0.8381 - val_loss: 0.3617 - val_accuracy: 0.8373 - val_f1: 0.8373
Epoch 2/10
20000/20000 [==============================] - 59s 3ms/sample - loss: 0.3588 - accuracy: 0.8420 - f1: 0.8420 - val_loss: 0.3546 - val_accuracy: 0.8440 - val_f1: 0.8440
Epoch 3/10
20000/20000 [==============================] - 55s 3ms/sample - loss: 0.3559 - accuracy: 0.8455 - f1: 0.8456 - val_loss: 0.3543 - val_accuracy: 0.8472 - val_f1: 0.8473
Epoch 4/10
20000/20000 [==============================] - 59s 3ms/sample - loss: 0.3429 - accuracy: 0.8502 - f1: 0.8502 - val_loss: 0.3564 - val_accuracy: 0.8474 - val_f1: 0.8475
Epoch 5/10
20000/20000 [==============================] - 57s 3ms/sample - loss: 0.3371 - accuracy: 0.8544 - f1: 0.8544 - val_loss: 0.3593 - val_accuracy: 0.8463 - val_f1: 0.8463
Epoch 6/10
20000/20000 [==============================] 

# observations:
In preprocessing 

1. apply spacing to all punct. except . increase the coverage.
2. correcting the spellings also helps.
3. recoving the html tags
4. also stemming increaes word coverage but this stem shoudnt be applied while preprocessing but while
   finding the word embedding.
   
   explaination. Stemming might help find word embeddings of words found in oov. but apply it at the 
   early preprcess steps is a bad idea, as it might stem other words which were present in the embedding 
   matrix but their stemming is not.

5. another important thing is taking care of spacing 
   in our approach this is taken care by the correction method.but we could also design an regix to care of this
   in the future.


Our current approach 



In Modeling 
1. setting training to true reduces the number of epochs 


# Simple Modification and Lr Scheduling 

In [ ]:
#LSTM netrow
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

def get_model2(vocab_size,weights = None,dim = 128,train = False,maxlen = 100):
  

    model = Sequential()
    model.add(Embedding(vocab_size,
                      dim,
                      weights= weights,
                      input_length=maxlen,
                      trainable=train))

    model.add(SpatialDropout1D(0.3))
    model.add(LSTM(300,return_sequences = True))
    model.add(GlobalMaxPooling1D())
#     model.add(Dense(1024, activation='sigmoid'))
#     model.add(Dropout(0.8))

    model.add(Dense(1024, activation='sigmoid'))
    model.add(Dropout(0.8))

    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(learning_rate = 0.001),
                  metrics = ['accuracy',f1])

    return model





In [ ]:
model  = get_model2(vocab_size,weights = [weights], dim = 300,train = True)
model.summary()
history1 = model.fit(trainx,trainy,
                    epochs = 5,
                    batch_size = 100,
                    validation_data=(testx,testy),
                
                    )

In [ ]:
from clr import CyclicLR


step_size = len(train_dataset) // 100

clr = CyclicLR(base_lr=0.001, max_lr=0.001,
                                step_size= step_size, mode='triangular')


step_size

In [ ]:
model  = get_model2(vocab_size,weights = [weights], dim = 300,train = True)
model.summary()
history1 = model.fit(trainx,trainy,
                    epochs = 5,
                    batch_size = 100,
                    validation_data=(testx,testy),
                    callbacks = [clr]
                    )

In [141]:
train_dataset.head()

,id,sentiment,review,processed
0,5814_8,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",the classic war of the worlds by timothy hi...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager nicholas bell ...
3,3630_4,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious 8...


In [143]:
test_dataset.head()

,id,review,sentiment,processed
0,12311_10,Naturally in a film who's main themes are of m...,1,naturally in a film who ' s main themes are of...
1,8348_2,This movie is a disaster within a disaster fil...,0,this movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i...",0,all in all this is a movie for kids we saw it...
3,7186_2,Afraid of the Dark left me with the impression...,0,afraid of the dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...,1,a very accurate depiction of small time mob li...


In [144]:
import pickle as pk


pk.dump(train_dataset,open('train_dataset','wb'))

pk.dump(test_dataset,open('test_dataset','wb'))
